In [ ]:
import os
import random
from datetime import datetime

now = datetime.now()

# Folder name to save registration transforms
resultsFolder = "results/" + now.strftime("%Y_%m_%d_%H_%M_%S")
print("Results folder name:", resultsFolder)

path = "./Dataset/data"
labelPath = "Dataset/labels"

data = os.listdir(path)
datasetLen = len(data)

# random shuffle data
random.shuffle(data)

# spilt dataset
trainData = data[:int(datasetLen * 0.7)]
testData = data[int(datasetLen * 0.7):]

"""Write transforms."""
from pathlib import Path

Path(f"./{resultsFolder}").mkdir(parents=True, exist_ok=True)

print("Saving training data names.")
file = open(f"./{resultsFolder}/trainData.txt", "w") 
for i, name in enumerate(trainData):
    # print(f"\tSaving {name}. {i+1}/{len(trainData)}", end='\x1b[1K\r')
    print(f"\tSaving {name}. {i+1}/{len(trainData)}")
    file.write(name+"\n")
# print()
file.close()

print("Saving testData names.")
file = open(f"./{resultsFolder}/testData.txt", "w") 
for i, name in enumerate(testData):
    # print(f"\tSaving {name}. {i+1}/{len(testData)}", end='\x1b[1K\r')
    print(f"\tSaving {name}. {i+1}/{len(testData)}")
    file.write(name+"\n")
# print()
file.close()

In [ ]:
# Tests
atlasesNums = []
P = []
N = []

In [ ]:
import SimpleITK as sitk
import numpy as np
from utils import *

transforms = []
mseList = []
P = [3, 3, 3]
N = [5, 5, 5]

# Read fixed image
pathName = os.path.join(path, testData[0])
fixedImage = sitk.CurvatureFlow(sitk.ReadImage(pathName), timeStep = 0.04, numberOfIterations = 10)


for i, f in enumerate(trainData[:1]): # for test
# for i, f in enumerate(trainData):
    # print(f'Registration of {f}. {i+1}/{len(trainData)}', end='\x1b[1K\r')
    print(f'Registration of {f}. {i+1}/{len(trainData)}')
    
    pathName = os.path.join(path, f)
                  
    # Histogram matching
    image = sitk.HistogramMatching(sitk.CurvatureFlow(sitk.ReadImage(pathName), 
                                                      timeStep = 0.04, 
                                                      numberOfIterations = 10),
                                   fixedImage)

    label = sitk.ReadImage(os.path.join(labelPath, f))

    t = registration(fixedImage, image, label)

    # add transform to the list
    transforms.append(t)

    # add mse to the list
    mseList.append(mse3D(fixedImage, resampleImage(image, fixedImage, t)))
    
# sort mseList
mseListSorted = sorted(enumerate(mseList), key=lambda x: x[1])

# pick the desired number of atlases
atlasesNum = 1
minx = miny = minz = 10000
maxx = maxy = maxz = 0
images = []
labels = []

###############################################################################
# Read atlases and labels and tranform them
###############################################################################
for i in range(atlasesNum):
    idx = mseListSorted[i][0]
    t = transforms[idx]
    f = trainData[idx]
    pathName = os.path.join(path, f)
    # print(f'Reading {f}. {i+1}/{atlasesNum}', end='\x1b[1K\r')
    print(f'Reading {f}. {i+1}/{atlasesNum}')
    
    # Histogram matching
    images.append(sitk.GetArrayFromImage(resampleImage(
                sitk.HistogramMatching(
                    sitk.CurvatureFlow(sitk.ReadImage(pathName), 
                                       timeStep = 0.04, 
                                       numberOfIterations = 10),
                                    fixedImage), 
                  fixedImage, t)))
    
    label = sitk.ReadImage(os.path.join(labelPath, f))
    label = sitk.GetArrayFromImage(resampleSegmentation(label, fixedImage, t))
    labels.append(label)
    
    idxs = np.nonzero(label != 0)
    minx = min([minx, min(idxs[0])])
    maxx = max([maxx, max(idxs[0])])
    miny = min([miny, min(idxs[1])])
    maxy = max([maxy, max(idxs[1])])
    minz = min([minz, min(idxs[2])])
    maxz = max([maxz, max(idxs[2])])
    
    
###############################################################################
# Calculate the desired shape
###############################################################################
shape, copyShape, offset, length = calculateCropShape(images, P, N, minx, maxx, miny, maxy, minz, maxz)

###############################################################################
# crop images and labels
###############################################################################
for i in range(len(images)):
    newImage = np.zeros((shape[0][1] - shape[0][0] + 1, 
                         shape[1][1] - shape[1][0] + 1, 
                         shape[2][1] - shape[2][0] + 1
                        ), dtype="uint16", order="C")
    newImage[offset[0]:offset[0]+length[0], 
             offset[1]:offset[1]+length[1], 
             offset[2]:offset[2]+length[2]] = images[i][copyShape[0][0]:copyShape[0][1]+1,
                                                        copyShape[1][0]:copyShape[1][1]+1,
                                                        copyShape[2][0]:copyShape[2][1]+1,]
    images[i] = newImage

    newLabels = np.zeros((shape[0][1] - shape[0][0] + 1, 
                          shape[1][1] - shape[1][0] + 1, 
                          shape[2][1] - shape[2][0] + 1,
                         ), dtype="uint8", order="C")
    newLabels[offset[0]:offset[0]+length[0], 
              offset[1]:offset[1]+length[1], 
              offset[2]:offset[2]+length[2]] = labels[i][copyShape[0][0]:copyShape[0][1]+1,
                                                         copyShape[1][0]:copyShape[1][1]+1,
                                                         copyShape[2][0]:copyShape[2][1]+1,]
    labels[i] = newLabels

fixedImage = sitk.GetArrayFromImage(fixedImage)
newImage = np.zeros((shape[0][1] - shape[0][0] + 1, 
                     shape[1][1] - shape[1][0] + 1, 
                     shape[2][1] - shape[2][0] + 1
                    ), dtype="uint16", order="C")
newImage[offset[0]:offset[0]+length[0], 
         offset[1]:offset[1]+length[1], 
         offset[2]:offset[2]+length[2]] = fixedImage[copyShape[0][0]:copyShape[0][1]+1,
                                                     copyShape[1][0]:copyShape[1][1]+1,
                                                     copyShape[2][0]:copyShape[2][1]+1,]
fixedImage = newImage

###############################################################################
# Make to the proper type
###############################################################################
# print("Fixing data types.", end='\x1b[1K\r')
print("Fixing data types.")
images = np.array(images, order='C')
labels = np.array(labels, order='C')
P = np.array(P, dtype=np.int32)
N = np.array(N, dtype=np.int32)

###############################################################################
# Perform segmentation
###############################################################################
from cGen import cGen
print("Running segmentation.")
'''
segmentation = cGen.applySPBM(fixedImage, images, labels, 3, P, N, lassoTol=0.00001, 
                              lassoMaxIter=1e4, verboseX=True,  verboseY=False, 
                              xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=60, zmax=61)
'''
segmentationSPBM, segmentationSRC = cGen.applySPBMandSRCSpams(fixedImage, images, 
                              labels, 3, P, N, lassoTol=0.00001, 
                              lassoMaxIter=1e4, verboseX=True,  verboseY=False, 
                              xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=-1, zmax=-1)
                              # xmin=10, xmax=11, ymin=10, ymax=11, zmin=10, zmax=11)
    
###############################################################################
# Save segmentation results
###############################################################################

# print("Saving results.", end='\x1b[1K\r')
print("Saving results.")
saveSegmentation(segmentationSPBM,
                 os.path.join(path, testData[0]), 
                 os.path.join(resultsFolder, testData[0]) + "SPBM.mhd", 
                 copyShape, 
                 offset,
                 length,
                 verbose=False)

saveSegmentation(segmentationSRC,
                 os.path.join(path, testData[0]), 
                 os.path.join(resultsFolder, testData[0]) + "SRC.mhd", 
                 copyShape, 
                 offset,
                 length,
                 verbose=False)

###############################################################################
# Save dice index
###############################################################################

originalLabel = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(labelPath, testData[0])))
diceFile = open(f"./{resultsFolder}/dice.txt", "w") 

translatedSegmentationSPBM = translateToOriginal(segmentationSPBM, 
                                                 os.path.join(path, testData[0]), 
                                                 copyShape, 
                                                 offset, 
                                                 length)

diceFile.write("SPBM: " + str(Dice(originalLabel, translatedSegmentationSPBM, 3)) + "\n")

translatedSegmentationSRC = translateToOriginal(segmentationSRC, 
                                                os.path.join(path, testData[0]), 
                                                copyShape, 
                                                offset, 
                                                length)

diceFile.write("SRC: " + str(Dice(originalLabel, translatedSegmentationSRC, 3)) + "\n")

diceFile.close()

In [ ]:
####################################################
# END OF NORMAL CODE 
####################################################
import sys
sys.exit()

In [ ]:
print(len(fixedImage))
print(len(images))
print(len(labels))
print(fixedImage.shape)
print(images.shape)
print(labels.shape)
print(fixedImage.flags)
print(images.flags)
print(labels.flags)

In [ ]:
# B.flags['F_CONTIGUOUS']
# B = np.asfortranarray([1,2,3,4,5], dtype='single')
B = np.zeros((5,1), dtype='single', order='F')
print(B.shape)
# B = np.asfortranarray(B[:, None])
print(B)
print(B[:,0])
print(B[:, None])

In [ ]:
B[:,0]

In [ ]:
B = np.asfortranarray(B[:, None])
print(B.flags['F_CONTIGUOUS'])
print(B.shape)

In [ ]:
B = np.asfortranarray(B[:, None])
B.flags['F_CONTIGUOUS']

In [ ]:
segmentationSPBM, segmentationSRC = cGen.applySPBMandSRCSpams(fixedImage, images, labels, 3, P, N, lassoTol=0.00001, 
                              lassoMaxIter=1e4, verboseX=True,  verboseY=False, 
                              xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=-1, zmax=-1)

In [ ]:
from PIL import Image

def showImg_(img, z=60):
    a = sitk.GetArrayFromImage(img[z,:,:])
    disImg = Image.fromarray(np.interp(a, (a.min(), a.max()), (0, 255)).astype('uint8'))
    disImg.show()
    
def showImg2_(img, z=60):
    disImg = Image.fromarray(sitk.GetArrayFromImage(img[z,:,:]).astype('uint8'))
    disImg.show()
    
def showImg3_(img, z=60):
    a = sitk.GetArrayFromImage(img[z,:,:])
    disImg = image.fromarray(np.interp(a, (0, 5000), (0, 255)).astype('uint8'))
    disImg.show()
    
def showImg4_(img, z=60):
    img2 = Image.fromarray(sitk.GetArrayFromImage(img[z,:,:])*60, 'L')
    img2.show()

In [ ]:
showImg2_(fixedImage, 60)
showImg2_(images[0], 60)
showImg2_(images[1], 60)

In [ ]:
l = sitk.GetArrayFromImage(label)
l[l == 4] = 0
mask = sitk.GetImageFromArray(l)

showImg4_(mask, 100)


In [ ]:
idx = 0
f = trainData[idx]
pathName = os.path.join(path, f)
image = sitk.HistogramMatching(sitk.CurvatureFlow(sitk.ReadImage(pathName), 
                                                      timeStep = 0.04, 
                                                      numberOfIterations = 10),
                               fixedImage)
'''
label = sitk.ReadImage(os.path.join(labelPath, f))
# Create the mask
l = sitk.GetArrayFromImage(label)
l[l>1] = 1
mask = sitk.GetImageFromArray(l)
mask.CopyInformation(label)
'''
    
elastixImageFilter = sitk.ElastixImageFilter()
elastixImageFilter.SetFixedImage(fixedImage)
elastixImageFilter.SetMovingImage(image)
# elastixImageFilter.AddMovingMask(mask)
elastixImageFilter.SetParameterMap(sitk.GetDefaultParameterMap("affine"))
elastixImageFilter.Execute()

In [ ]:
resultImage2 = elastixImageFilter.GetResultImage()
transformParameterMap = elastixImageFilter.GetTransformParameterMap()

In [ ]:
sitk.PrintParameterMap(elastixImageFilter.GetParameterMap())

In [ ]:
help(sitk.ElastixImageFilter)

In [ ]:
print(transformParameterMap)

In [ ]:
showImg2_(fixedImage, 60)
showImg2_(images[0], 60)
showImg2_(resultImage, 60)

In [ ]:
showImg_(fixedImage, 60)
showImg_(images[1], 60)
showImg_(resultImage, 60)

In [ ]:
help(sitk.GetImageFromArray)

In [ ]:

labels = np.array(labels, order='C', dtype=np.uint8)

segmentationSPBM, segmentationSRC = cGen.applySPBMandSRCSpams(fixedImage, images, 
                              labels, 3, P, N, lassoTol=0.00001, 
                              lassoMaxIter=1e4, verboseX=True,  verboseY=False, 
                              xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=-1, zmax=-1)

In [ ]:
segmentationSPBM, segmentationSRC = cGen.applySPBMandSRCSpams(fixedImage,
                                                         images, 
                              np.ones(labels.shape, dtype=np.uint8, order='C')+50, 3, P, N, lassoTol=0.00001, 
                              lassoMaxIter=1e4, verboseX=True,  verboseY=False, 
                              xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=-1, zmax=-1)

In [ ]:
print(fixedImage.shape)
print(fixedImage.dtype)
print(type(fixedImage))
print(images.shape)
print(images.dtype)
print(type(images))
print(labels.shape)
print(labels.dtype)
print(type(labels))

In [1]:
from cGen import cGen
import numpy as np

P = [3, 3, 3]
N = [5, 5, 5]
P = np.array(P, dtype=np.int32)
N = np.array(N, dtype=np.int32)

ii = np.array(np.random.randint(0, high=500, size=(390, 276, 129)), dtype=np.uint16, order='C')
i = np.array(np.random.randint(0, high=500, size=(1, 390, 276, 129)), dtype=np.uint16, order='C')
l = np.array(np.random.randint(0, high=3, size=(1, 390, 276, 129)), dtype=np.uint8, order='C')
segmentationSPBM, segmentationSRC = cGen.applySPBMandSRCSpams(ii,
                                                              i, 
                                                              l, 3, P, N, lassoTol=0.00001, 
                              lassoMaxIter=1e4, verboseX=True,  verboseY=False, 
                              xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=-1, zmax=-1)

X: 3 	Time: 191.71075296401978


KeyboardInterrupt: 

In [ ]:
import spams

A = np.zeros(shape=(20, 10), dtype=np.single, order='F')
B = np.zeros(shape=(20, 1), dtype=np.single, order='F')
# B = np.zeros(shape=(20, 1), dtype=np.single, order='F')


alpha = spams.lasso(B, A, 
               return_reg_path = False, 
               lambda1 = 0.1, 
               lambda2 = 0.,
               pos = True,
               mode = 2,
               numThreads = -1,
              )


In [ ]:
alpha.toarray()[:,0]